In [5]:
!pip install numpy
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreaceated sklearn

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Task 1

In [9]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./online_shoppers_intention.csv"))
frame.describe(include='all') # 18 columns, including 2 categorical: Month & VisitorType, and two boolean: Weekend & Revenue

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330,12330.000000,12330.000000,12330.000000,12330.000000,12330,12330,12330
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,3,2,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,May,NaN,NaN,NaN,NaN,Returning_Visitor,False,False
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3364,NaN,NaN,NaN,NaN,10551,9462,10422
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,NaN,2.124006,2.357097,3.147364,4.069586,NaN,NaN,NaN
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,NaN,0.911325,1.717277,2.401591,4.025169,NaN,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,NaN,2.000000,2.000000,1.000000,2.000000,NaN,NaN,NaN
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,NaN,2.000000,2.000000,3.000000,2.000000,NaN,NaN,NaN
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,NaN,3.000000,2.000000,4.000000,4.000000,NaN,NaN,NaN


Verhaaltje van ~1min

In [10]:
#visualisation
import plotly.express as px
fig = px.line(frame, x='Browser', y=frame.columns)
fig.show()

ValueError: Plotly Express cannot process wide-form data with columns of different type.

Task 2

In [11]:
#Prepositioning op basis van algoritmes in task 3
# Preprocess using dropna AND change all string types to an integer
#TODO Jan & Apr aren't in dataset, maybe remove from this function
def Month_to_num(x):
    months = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
        "May": 5, "June": 6, "Jul": 7, "Aug": 8,
        "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12, "Returning_Visitor": 1, "Old_Visitor": 2, "New_Visitor": 3, "Other": 4}
    if x not in months:
        raise ValueError(f"Invalid month string: {x}")
    return months[x]

addlist = []
preprocessed = (frame.copy().dropna())
for i,j in enumerate(preprocessed['Month']):
    addlist.append(Month_to_num(j))
preprocessed['Month'] = addlist

addlist = []
for i,j in enumerate(preprocessed['VisitorType']):
    addlist.append(Month_to_num(j))
preprocessed['VisitorType'] = addlist

for i in preprocessed.columns:
    preprocessed[i] = pd.to_numeric(preprocessed[i], errors='raise')

print(preprocessed)



       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

Task 3

In [ ]:
# affinity propagation clustering - aan de hand van documentatie - NU HEEL SLOOM
from sklearn.cluster import AffinityPropagation, DBSCAN
Data = preprocessed.copy()
#Gebruik van sample maakt het proces sneller
Sample = Data.sample(n=1000)
clustering = AffinityPropagation(random_state=5).fit(Sample)
AffinityPropagation(random_state=24)

labels = clustering.labels_
cluster_centers = clustering.cluster_centers_
n_clusters = len(cluster_centers)

print("Number of clusters:", n_clusters)
print("Cluster centers:\n", cluster_centers)


# DBSCAN clustering
#TODO werkt maar de label -1 laat zien dat er een hoop data punten in ruis categorie vallen...
clustering2 = DBSCAN(eps=3).fit(Data)
labels2 = clustering2.labels_
print("Labels", labels2[:50])


# Birch clustering

c:\Users\tibor\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\cluster\_affinity_propagation.py:140: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


Number of clusters: 338
Cluster centers:
 [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 3. 0. 0.]]
Labels [ 0 13 -1 -1 -1 -1  0  0  1 -1 -1 -1 -1 -1 -1 -1  0 -1  2 -1 -1  0 -1 -1
  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  4 -1 -1  7 -1  2 -1 -1 -1 10
 -1  0]


Task 4

In [30]:
# manual Silhouette score

# Davies Buildin Score

# calinski-harabasz index

Task 5 ALL MANUAL


In [31]:
# Euclidian distance

# Manhattan distance

# Cosine similarity

# Distances effect on dbscan